In [1]:
import pandas as pd 
import numpy as np
from numpy import loadtxt
import warnings

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt

In [2]:
realdata = pd.read_csv(f"C:\\Users\\namil\\Documents\\stmi-lab-namila\\allmetrics1")
savedval = 0.111

def basemodel(participant, arr, start, end, topredict, removesnacks, nodinners):
    global realdata
    global savedval
    
    test = realdata
    
    #remove specified entries
    if nodinners:
        test = test[test['Meal Type'] != "dinner"]
    
    if removesnacks:
        if(len(test[test['snack in 3hrs'] == True]) == 0):
            arr.append(savedval)
            return savedval
        else:
            test = test[test['snack in 3hrs'] == False]

    valid = test[test['dexcom 3hr auc'] != "error"]
    valid = valid[valid['mets 3 hr auc'] != "error"]
    
    #for predicting macros
    #X = valid.loc[:, 'peakheight':'mets 3 hr avg'].to_numpy()
    X = valid.loc[:, start:end].to_numpy()
    #for predicting ppgr 
    #X = valid.iloc[:, 10:18].to_numpy()
    #X = valid.loc[:, 'mets 3 hr auc':'fat'].to_numpy()
    #X = valid.loc[:, 'calories':'fat'].to_numpy()
    
    Y = valid[topredict].to_numpy().astype(float)
    warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


    #split into train and test sets
    seed = 7
    test_size = 0.25
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    # HP TUNING
    # Define the parameter grid
    param_grid = {'alpha': [0.0, 0.5, 1.0, 2.0],
                  'learning_rate': [0.3, 0.1, 0.01, 0.001],  # Learning rate of the model
                  'max_depth': [2, 3, 4, 5, 6],  # Maximum depth of each tree
                  'n_estimators': [100, 200, 250],  # Number of trees in the ensemble
                  }
    # Define the XGBoost regressor
    model = XGBRegressor()
    # Set up the grid search cross-validation
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    # Perform the grid search
    grid_search.fit(X_train, y_train)
    # print(grid_search.best_params_)
    
    model = XGBRegressor(alpha = grid_search.best_params_['alpha'],
                        learning_rate = grid_search.best_params_['learning_rate'],
                        max_depth = grid_search.best_params_['max_depth'],
                        n_estimators = grid_search.best_params_['n_estimators'])
    #make model
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

#     plt.plot(y_test)
#     plt.plot(prediction)
#     plt.show()
    
#     # Fits the explainer
#     explainer = shap.Explainer(model.predict, X_test)
#     # Calculates the SHAP values - It takes some time
#     shap_values = explainer(X_train)
#     #shap.summary_plot(shap_values)
#     shap.plots.bar(shap_values)
    
    #sicong's normalized rmse
    errors = ((y_test - prediction)/y_test)**2
    rmsre = (np.average(errors))**.5
    print("RMSRE:", rmsre)
    
    arr.append(rmsre)
    return rmsre
    
def runfour(errors, start, end, topredict):
    global savedval
    savedval = basemodel(x, errors[0], start, end, topredict, 0, 1)
    basemodel(x, errors[1], start, end, topredict, 1, 1)
    savedval = basemodel(x, errors[2], start, end, topredict,  0, 0)
    basemodel(x, errors[3], start, end, topredict, 1, 0)

In [ ]:
participants = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29]
#participants = [26]
inputerrors = [[],[],[],[]]
inputerrors2 = [[],[],[],[]]

predpeakh = [[],[],[],[]]
predauc = [[],[],[],[]]
prediauc = [[],[],[],[]]
predpt = [[],[],[],[]]
predpd = [[],[],[],[]]
sbg = [[],[],[],[]]

for x in participants:
    print(x)
    
    global realdata
    realdata = pd.read_csv(f"C:\\Users\\namil\\Documents\\stmi-lab-namila\\allmetrics{x}")
    
    #predicting macros
    start = 'peakheight'
    end = 'mets 3 hr avg'

    runfour(inputerrors, start, end, 'carbs')
    runfour(inputerrors2, start, end, 'calories')
    
    #predicting ppgr
    start = 'mets 3 hr auc'
    end = 'fat'

#     runfour(predpeakh, start, end, 'peakheight')
#     runfour(predauc, start, end, 'dexcom 3hr auc')

    runfour(prediauc, start, end, 'iauc')
    runfour(predpt, start, end, 'peaktime')
    runfour(predpd, start, end, 'peakduration_40')
    runfour(sbg, start, end, 'startbg')
    
df = pd.DataFrame({'participants': participants, 
                  'carbs bl raw rmsre': inputerrors[0], 'carbs bl removed rmsre': inputerrors[1],
                  'carbs bld raw rmsre': inputerrors[2], 'carbs bld removed rmsre': inputerrors[3],
                  'calories bl raw rmsre': inputerrors2[0], 'calories bl removed rmsre': inputerrors2[1],
                  'calories bld raw rmsre': inputerrors2[2], 'calories bld removed rmsre': inputerrors2[3],
#                   'ph bl raw rmsre': predpeakh[0], 'ph bl removed rmsre': predpeakh[1],
#                   'ph bld raw rmsre': predpeakh[2], 'ph bld removed rmsre': predpeakh[3],
#                   'auc bl raw rmsre': predauc[0], 'auc bl removed rmsre': predauc[1],
#                   'auc bld raw rmsre': predauc[2], 'auc bld removed rmsre': predauc[3],
                  'iauc bl raw rmsre': prediauc[0], 'iauc bl removed rmsre': prediauc[1],
                  'iauc bld raw rmsre': prediauc[2], 'iauc bld removed rmsre': prediauc[3],
                  'pt bl raw rmsre': predpt[0], 'pt bl removed rmsre': predpt[1],
                  'pt bld raw rmsre': predpt[2], 'pt bld removed rmsre': predpt[3],
                  'pd40 bl raw rmsre': predpd[0], 'pd40 bl removed rmsre': predpd[1],
                  'pd40 bld raw rmsre': predpd[2], 'pd40 bld removed rmsre': predpd[3],
                  'sbg bl raw rmsre': sbg[0], 'sbg bl removed rmsre': sbg[1],
                  'sbg bld raw rmsre': sbg[2], 'sbg bld removed rmsre': sbg[3]
                  })
df.to_csv("tunedwithhractivity")

#print(f"carbs raw error: {np.average(rawerrors)} \ncarbs removed error: {np.average(removederrors)} \ncarbs bld raw error: {np.average(drawerrors)}\ncarbs bld removed: {np.average(dremovederrors)}")
print("done!")

1
RMSRE: 0.259844892431576
RMSRE: 0.9439792299990142
RMSRE: 0.34128157425498384
RMSRE: 0.4499502108014379
RMSRE: 0.3368341540631005
RMSRE: 0.5629649470576072
RMSRE: 0.40205751754248176
RMSRE: 0.968530274608606
RMSRE: 0.8924625714074272
RMSRE: 1.1287822183338452
RMSRE: 0.6992281583251939
RMSRE: 0.9889469893079897
RMSRE: 2.068343761219719
RMSRE: 0.522266579317998
RMSRE: 3.655499642040646
RMSRE: 1.2101815192938432
RMSRE: 0.23172324274876102
RMSRE: 0.2026621144694682
RMSRE: 0.2360341818876964
RMSRE: 0.09725107164713535
RMSRE: 0.14436381360426342
RMSRE: 0.09989101946810217
RMSRE: 0.07859068417077136
RMSRE: 0.11939746610581231
2
RMSRE: 0.40911425843121063
RMSRE: 1.4403682394221002
RMSRE: 0.49131653416264376
RMSRE: 0.9428522549580918
RMSRE: 0.48343211657726054
RMSRE: 0.9222711842986041
RMSRE: 0.4155407241995169
RMSRE: 1.0291119108335334
RMSRE: 1.4489655710234057
RMSRE: 0.49524144532872943
RMSRE: 1.599719939890756
RMSRE: 0.5788324716474067
RMSRE: 1.9440685946102318
RMSRE: 0.6020768136380867
RM

RMSRE: 0.47294132200485706
RMSRE: 1.5324395975340215
RMSRE: 1.4237172270795442
RMSRE: 1.4849027402055655
RMSRE: 0.07790016363861117
RMSRE: 0.03718764094672546
RMSRE: 0.03575565528240845
RMSRE: 0.055411003272332456
RMSRE: 0.11421022252771462
RMSRE: 0.1219800712409387
RMSRE: 0.0953438122403995
RMSRE: 0.07303906587729468
18
RMSRE: 0.594912904275598
RMSRE: 0.8816773553637742
RMSRE: 1.6248666936160234
RMSRE: 0.36557379771764725
RMSRE: 1.1266357621199814
RMSRE: 0.9098455507500954
RMSRE: 0.8712545099531509
RMSRE: 0.8377413482551554
RMSRE: 0.8065440620400481
RMSRE: 9.151406572466994
RMSRE: 6.602718772018928
RMSRE: 5.799689133483023
RMSRE: 0.15191381997556505
RMSRE: 0.19101677399745595
RMSRE: 0.12714371560514995
RMSRE: 0.2585942028022004
RMSRE: 0.19913833565882166
RMSRE: 0.2438660679974733
19
RMSRE: 0.7528924387927977
RMSRE: 0.49257502797775604
RMSRE: 10.512478184953501
RMSRE: 9.238547790478012
RMSRE: 0.3832537163176469
RMSRE: 0.35431716913639594
RMSRE: 2.8900851939100867
RMSRE: 2.3947502622386